In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import urllib
import urllib.request
from splinter import Browser
import pandas as pd
import re

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
nasa_db = client.nasa_db
nasa_collection = nasa_db.articles
space_db = client.space_db
space_collection = space_db.articles
weather_db = client.weather_db
weather_collection = weather_db.articles
facts_db = client.facts_db
facts_collection = facts_db.articles
astro_db = client.astro_db
astro_collection = astro_db.articles

NASA Page Articles
* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."

In [14]:
# URL of page to be scraped
nasa_url = 'https://mars.nasa.gov/news/'
#nasa_page = urllib.request.urlopen (nasa_url)
browser.visit(nasa_url)
browser.is_element_present_by_css('ul.item_list li.slide', wait_time=5)

True

In [15]:
html=browser.html
nasa_soup=BeautifulSoup(html, 'html.parser')

#nasa_soup

In [16]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
#nasa_results = nasa_soup.find_all('div', class_='')
nasa_results = nasa_soup.select_one("ul.item_list li.slide")
#nasa_results

In [17]:
#pull news data
title = nasa_results.find('div', class_='content_title').get_text()
thread = nasa_results.find('div', class_='article_teaser_body').get_text()

print(title)
print(thread)

Three New Views of Mars' Moon Phobos
Taken with the infrared camera aboard NASA's Odyssey orbiter, they reveal temperature variations on the small moon as it drifts into and out of Mars’ shadow.


Space Imagaes of Mars
* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [18]:
space_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
space_page = urllib.request.urlopen (space_url)
space_soup=BeautifulSoup(space_page, 'html.parser')
#space_soup

In [19]:
space_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(space_url)

In [20]:
image=browser.find_by_id('full_image')
image.click()

In [21]:
browser.is_element_present_by_text("more info", wait_time=1)
more_info_link= browser.find_link_by_partial_text("more info")
more_info_link.click()

C:\Users\rsand\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [22]:
space_html = browser.html
space_soup = BeautifulSoup(space_html, 'html.parser')


In [23]:
feature_image=space_soup.select_one("figure.lede a img").get("src")
feature_image

'/spaceimages/images/largesize/PIA18280_hires.jpg'

In [24]:
featured_image_url = 'https://www.jpl.nasa.gov' + feature_image 
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18280_hires.jpg'

Mars Weather on Twitter
* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.
* **Note: Be sure you are not signed in to twitter, or scraping may become more difficult.**
* **Note: Twitter frequently changes how information is presented on their website. If you are having difficulty getting the correct html tag data, consider researching Regular Expression Patterns and how they can be used in combination with the .find() method.**

In [25]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [26]:
# weather_page = urllib.request.urlopen (weather_url)
weather_page = browser.html
weather_soup=BeautifulSoup(weather_page, 'html.parser')
#weather_soup


In [27]:
weather_results = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
weather_results

In [28]:
pattern = re.compile(r"sol")
pattern
mars_weather = weather_soup.find('span', text=pattern).text
mars_weather

'InSight sol 546 (2020-06-09) low -92.7ºC (-134.9ºF) high -6.6ºC (20.1ºF)\nwinds from the WNW at 6.4 m/s (14.4 mph) gusting to 25.1 m/s (56.2 mph)\npressure at 7.40 hPa'

Mars Facts
*Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

*Use Pandas to convert the data to a HTML table string.

In [29]:
facts_url = 'https://space-facts.com/mars/'
facts_page = urllib.request.urlopen (facts_url)
facts_soup=BeautifulSoup(facts_page, 'html.parser')
#facts_soup

In [30]:
facts_results = facts_soup.find_all('tr', class_='row')
facts_results

[]

In [31]:
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
#print(mars_df)
mars_df.columns=["Desc.", "Value"]
mars_df.set_index("Desc.", inplace=True)
mars_df

,Value
Desc.,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


Mars Hemispheres
* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},

In [32]:
base_hemisphere_url = "https://astrogeology.usgs.gov"
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [33]:
html = browser.html
astro_soup = BeautifulSoup(html, 'html.parser')

hemisphere_names = []

# Search for the names of all four hemispheres
results = astro_soup.find_all('div', class_="collapsible results")
hemispheres = results[0].find_all('h3')
hemispheres

# # Get text and store in list
for i,name in enumerate(hemispheres):
    mars_dict={}
    mars_dict["title"]= name.text
    browser.find_by_css('a.product-item h3')[i].click()
    sample = browser.find_link_by_text("Sample").first
    mars_dict["url"]= sample['href']
    hemisphere_names.append(mars_dict)
    browser.back()
    

hemisphere_names

C:\Users\rsand\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


[{'title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]